In [3]:
import numpy as np
import tensorflow as tf
from adult_modified import preprocess_adult_data
from sklearn import linear_model
import classifier as cl
import utils
import time
import multiprocessing as mp
import random
import matplotlib.pyplot as plt
import scipy
plt.ioff()


seed = 1
tf.random.set_seed(seed)
np.random.seed(seed)
dataset_orig_train, dataset_orig_test = preprocess_adult_data(seed = seed)

x_unprotected_train, x_protected_train = dataset_orig_train.features[:, :39], dataset_orig_train.features[:, 39:]
x_unprotected_test, x_protected_test = dataset_orig_test.features[:, :39], dataset_orig_test.features[:, 39:]
y_train, y_test = dataset_orig_train.labels.reshape((-1,)), dataset_orig_test.labels.reshape((-1,))





## Running linear regression to get sensetive directions 

protected_regression = linear_model.LinearRegression(fit_intercept = False)
protected_regression.fit(x_unprotected_train, x_protected_train)
sensetive_directions = protected_regression.coef_

sensetive_directions = scipy.linalg.orth(sensetive_directions.T).T
for i, s in enumerate(sensetive_directions):
    while np.linalg.norm(s) != 1:
        s = s/ np.linalg.norm(s)
    sensetive_directions[i] = s





unprotected_directions = utils.projection_matrix(sensetive_directions)
protected_directions = utils.projection_matrix2(sensetive_directions)



# Casing to tensor 
y_train, y_test = y_train.astype('int32'), y_test.astype('int32')
x_unprotected_train, x_unprotected_test = tf.cast(x_unprotected_train, dtype = tf.float32), tf.cast(x_unprotected_test, dtype = tf.float32)
y_train, y_test = tf.one_hot(y_train, 2), tf.one_hot(y_test, 2)
unprotected_directions = tf.cast(unprotected_directions, dtype = tf.float32)
protected_directions = tf.cast(protected_directions, dtype = tf.float32)
sensetive_directions = tf.cast(sensetive_directions, dtype = tf.float32)

init_graph = utils.ClassifierGraph(50, 2)
#graph = cl.Classifier(init_graph, x_unprotected_train, y_train, x_unprotected_test, y_test, num_steps = 10000) # use for unfair algo
graph = cl.Classifier(init_graph, utils.unprotected_direction(x_unprotected_train, sensetive_directions), 
                        y_train, num_steps = 1000) # for fair algo





Done step 200

Done step 400

Done step 600

Done step 800

Done step 1000



In [18]:
def sample_perturbation(data_point, regularizer = 1, learning_rate = 5e-4, num_steps = 10):
    x, y = data_point
    x = tf.reshape(x, (1, -1))
    y = tf.reshape(y, (1, -1))
    x_start = x
    x += tf.cast(np.random.normal(size=(1, 39)), dtype = tf.float32)*0.0001
    for _ in range(num_steps):
        with tf.GradientTape() as g:
            g.watch(x)
            prob = graph(utils.unprotected_direction(x, sensetive_directions))
            perturb = utils.unprotected_direction(x-x_start, sensetive_directions)
            print(perturb)
            print(prob)
            loss = utils.EntropyLoss(y, prob) / regularizer - tf.norm(perturb)
            print(loss)

        gradient = g.gradient(loss, x)
        print(gradient)
        x = x + learning_rate * utils.protected_direction(gradient, sensetive_directions)
        print('\n\n')

    print(utils.unprotected_direction(x - x_start, sensetive_directions))
    return_loss = utils.EntropyLoss(y, graph(utils.unprotected_direction(x, sensetive_directions)))\
         / utils.EntropyLoss(y, graph(utils.unprotected_direction(x_start, sensetive_directions)))
    
    return return_loss.numpy()




sample_perturbation((x_unprotected_test[0], y_test[0]))

tf.Tensor(
[[-6.1105337e-04  1.5960935e-03 -5.2998967e-05 -2.7767839e-04
   2.1537770e-04  1.0929690e-03 -8.6866418e-04 -1.9218880e-04
  -2.2069563e-03  1.9369621e-04 -2.7646561e-04  1.6209760e-03
   7.2462362e-04 -8.1873196e-04  1.1637608e-04  4.1847053e-04
   4.5037916e-04 -2.1742002e-04  2.2877075e-04  7.0644286e-04
  -1.2510102e-03 -3.7325284e-04 -9.7486982e-04 -4.1326639e-05
   6.9223274e-04  8.6040783e-04  8.6357111e-05 -8.4753346e-04
  -8.7926106e-04 -1.9072841e-03 -1.2648278e-03 -1.1846101e-03
  -1.9546049e-03  7.2706415e-04  1.5649612e-03 -1.1787373e-04
   3.3816003e-04  9.1592956e-04  6.9010665e-04]], shape=(1, 39), dtype=float32)
tf.Tensor([[0.9837024  0.01629762]], shape=(1, 2), dtype=float32)
tf.Tensor(0.01050831, shape=(), dtype=float32)
tf.Tensor(
[[ 0.10587641 -0.25642028  0.01742656  0.05006473 -0.03076017 -0.1844267
   0.14246532  0.01577964  0.37237212 -0.03047384  0.04833082 -0.26939702
  -0.12699428  0.14624937 -0.0188084  -0.06548437 -0.08688976  0.0342283
  -0.03

0.9991158